In [1]:
import numpy as np
import math 
import random
import time

## P20-EX1

In [8]:
f = lambda x:math.sqrt(1-x**2)
slices = [1000, 10000, 100000, 1000000]
ans = []
for slice in slices:
    sum = 0
    dx = 1.0 / slice
    for i in range(slice):
        sum += f(i * dx) * dx
    ans.append(sum * 4)
print(ans)


[3.143555466911028, 3.1417914776113167, 3.1416126164019564, 3.1415946524138207]


## P23-EX3

In [9]:
def F(a, b, c, d, n, f):
    assert a <= b and c <= d and n > 0
    sum = 0
    for i in range(n):
        x = np.random.uniform(a,b)
        y = np.random.uniform(c,d)
        if f(x) > y and y >= 0.:
            sum += 1
        elif f(x) <= y and y < 0.:
            sum -= 1
    return float(sum) / n * (b-a) * (d - c) 


In [22]:
f = lambda x:x**3-2
print (F(-1,2,-3,6,1000000,f))

-2.246265


## P36-EX5 估计整数子集 $1\sim n$的大小

In [5]:
def getK(X):
    S = []
    k = 0
    getUniformX = lambda :random.sample(X, 1)[0]
    a = getUniformX()
    while True:
        k += 1
        S.append(a)
        a = getUniformX()
        if a in S:
            break
    return k

def SetCount(X, times=1000):
    ave = 0.
    for _ in range(times):
        ave += getK(X)
    ave /= times
    return 2 * ave * ave / math.pi


In [10]:
N = [1, 5, 10, 50, 100, 500, 1000, 5000, 10000]
ans = []
for n in N:
    X = list(np.arange(1, n+1))
    ans.append(SetCount(X))
print(ans)

[0.6366197723675814, 4.033170241062842, 8.743613519917115, 45.252054507306816, 96.17330931009076, 498.8247366218286, 952.3260931302938, 5043.345149758812, 10044.185484054864]


## P67-EX7 搜索有序链表...

In [127]:
class OrderedList():
    def __init__(self, len=10000) -> None:
        self.len = len
        self.val = []
        self.next = []
        self.pre = []
        #生成测试数据
        for i in range(len):
            self.val.append(i)
            self.next.append(i+1)
            self.pre.append(i-1)
        self.next[-1] = 0
        self.pre[0] = len - 1
        self.head = 0
        #打乱数据
        self.shuffle()


    def shuffle(self):
        for i in range(self.len):
            r = np.random.randint(self.len)
            if self.head == i:
                self.head = r
            elif self.head == r:
                self.head = i

            self.val[i], self.val[r] = self.val[r], self.val[i]

            self.next[self.pre[i]], self.next[self.pre[r]] = r, i
            self.pre[i], self.pre[r] = self.pre[r], self.pre[i]
            self.pre[self.next[i]], self.pre[self.next[r]] = r, i
            self.next[i], self.next[r] = self.next[r], self.next[i]

    #serach函数，从ptr位置开始查找，返回可能对应x的ptr和查找计数
    def __call__(self, x, ptr=None):
        if ptr is None:
            ptr = self.head
        assert ptr < self.len
        cnt = 1
        while x > self.val[ptr]:
            cnt += 1
            ptr = self.next[ptr]
            assert ptr != self.head
        return ptr, cnt

    def getRandomPtr(self):
        return np.random.randint(self.len)
        


A算法，确定性算法

In [37]:
def A(x, ordered_list):
    return ordered_list(x)

D算法，时间O(n)的概率算法

In [10]:
def D(x, ordered_list):
    ptr = ordered_list.getRandomPtr()
    y = ordered_list.val[ptr]
    if y > x:
        ptr = None
    elif y < x:
        ptr = ordered_list.next[ptr]
    else:
        return ptr, 0
    return ordered_list(x, ptr)

B算法,平均时间为O $(\sqrt n)$ 的确定算法

In [11]:
def B(x, ordered_list):
    ptr  = ordered_list.head
    _max = ordered_list.val[ptr]
    batch = int(math.sqrt(ordered_list.len))

    for i in range(batch):
        y = ordered_list.val[i]
        if _max < y and y <= x:
            ptr = i
            _max = y

    return ordered_list(x, ptr)


C算法，Sherwood版的B算法

In [118]:
def C(x, ordered_list):
    ptr  = ordered_list.head
    _max = ordered_list.val[ptr]
    batch = int(math.sqrt(ordered_list.len))
    choices = np.random.choice(ordered_list.len, size=batch, replace=False)

    for choice in choices:
        y = ordered_list.val[choice]
        if _max < y and y <= x:
            ptr = choice
            _max = y

    return ordered_list(x, ptr)

性能测试

In [130]:
len = 10000
test = OrderedList(len)
test_times = 5000

worst = {"A":0, "B":0, "C":0, "D":0}
average = {"A":0., "B":0., "C":0., "D":0.}
alg_s = {"A":A, "B":B, "C":C, "D":D}

for name, alg in alg_s.items():
    for i in range(test_times):
        x = np.random.randint(len)
        ptr, cnt = alg(x, test)
        assert test.val[ptr] == x
        average[name] += cnt
        if cnt > worst[name]:
            worst[name] = cnt
    average[name] /= test_times


print(worst)
print(average)

{'A': 10000, 'B': 483, 'C': 892, 'D': 9785}
{'A': 4950.6982, 'B': 91.284, 'C': 98.3144, 'D': 3286.0576}


## P83-EX9

In [12]:
'''
target_k:目标构建的k皇后
k:当前已放置的安全的前k行皇后
col:存在皇后的列
diag45:当前45度对角线冲突表
diag135:当前135度对角线冲突表

cnt:访问节点计数
'''
def backtrace(target_k, col, diag45, diag135, row=0):
    if row == target_k:
        return True, 0
        
    cnt = 0
    for _col in range(target_k):
        if _col not in col and _col - row not in diag45 \
            and _col + row not in diag135:
            col.append(_col)
            diag45.append(_col - row)
            diag135.append(_col + row)
            #访问该节点，对应计数增1 
            cnt += 1
            isSuccess, _cnt = backtrace(target_k, col, diag45
                                        , diag135, row+1)
            #增加对应子节点的计数
            cnt += _cnt
            if isSuccess:
                return True, cnt
            col.pop()
            diag45.pop()
            diag135.pop()

    return False, cnt


In [231]:
"""
贪心的LV算法，前stepVegas皇后随机放置
"""
def QueensLV(target_k, stepVegas):
    col = []
    diag45 =[]
    diag135 = []
    row = 0
    k = 0
    nb = 0

    while True:
        nb = 0
        for _col in range(target_k):
            if _col not in col and _col - row not in diag45 \
                and _col + row not in diag135:
                nb += 1
                if np.random.randint(nb) == 0:
                    col_candidate = _col
        if nb > 0:
            col.append(col_candidate)
            diag45.append(col_candidate - k)
            diag135.append(col_candidate + k)
            k += 1
        if nb == 0 or k >= stepVegas:
            break
    
    if nb > 0:
        isSuccess, cnt = backtrace(target_k, col, diag45, diag135, k)
        return isSuccess, cnt + stepVegas
    else:
        return False, stepVegas



In [260]:
times = 1000
epoch = 100
ans = {}
for n in range(12, 21):
    best_s = 0.
    best_e = 0.
    best_t = math.inf
    bestStepVeags = 0
    bestSuccessRate = 0.
    for stepVegas in range(1, n + 1):
        successRate = 0.
        s = 0.
        e = 0.

        for _ in range(times):
            isSuccess, cnt = QueensLV(n, stepVegas)
            if isSuccess:
                successRate += 1
                s += cnt
            else:
                e += cnt
        
        successRate += 1e-6
        s /= successRate
        e /= times - successRate + 1e-6

        successRate /= times
        
        t = s + (1 - successRate) * e / successRate
        if t < best_t:
            best_t = t
            best_e = e
            best_s = s
            bestSuccessRate = successRate
            bestStepVeags = stepVegas
    ans[n] = (bestStepVeags, bestSuccessRate, best_s, best_e, best_t)

In [261]:
print(ans)

{12: (6, 0.535000001, 22.41495322913093, 25.335483870967742, 44.43551388827851), 13: (7, 0.495000001, 22.341414096279973, 24.59009900990099, 47.42828268279115), 14: (8, 0.461000001, 23.713665892161245, 23.122448979591837, 50.74837294171189), 15: (9, 0.425000001, 24.87294111794602, 22.768695652173914, 55.677646874243905), 16: (10, 0.371000001, 25.4312667778133, 21.620031796502385, 62.086253143648825), 17: (10, 0.563000001, 36.71047950850714, 39.09153318077803, 67.0532857794941), 18: (11, 0.485000001, 38.79999992, 35.0873786407767, 76.0577317295977), 19: (12, 0.405000001, 38.4617283000945, 30.707563025210085, 83.57530835979537), 20: (13, 0.358000001, 39.262569722730255, 25.774143302180686, 85.48323991268887)}
